In [1]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import Normalizer
from sklearn.pipeline import make_pipeline
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd 

In [4]:
df = pd.read_csv("../db/nutrition.csv")

In [5]:
df.head()

,NDB_No,Shrt_Desc,Water,Energy,Protein,Lipid_Total,Carbohydrate,Fiber,Sugar_Total,Calcium,...,Vitamin_K,FA_Sat_g,FA_Mono_g,FA_Poly_g,Cholestrol,Weight_grams,Weight_desc,GmWt_2,GmWt_Desc2,Refuse_Pct
0,1001,"BUTTER,WITH SALT",15.87,717,0.85,81.11,0.06,0.0,0.06,24,...,7.0,51.368,21.021,3.043,215,5.00,"1 pat, (1"" sq, 1/3"" high)",14.2,1 tbsp,0
1,1002,"BUTTER,WHIPPED,W/ SALT",16.72,718,0.49,78.30,2.87,0.0,0.06,23,...,4.6,45.390,19.874,3.331,225,3.80,"1 pat, (1"" sq, 1/3"" high)",9.4,1 tbsp,0
2,1003,"BUTTER OIL,ANHYDROUS",0.24,876,0.28,99.48,0.00,0.0,0.00,4,...,8.6,61.924,28.732,3.694,256,12.80,1 tbsp,205.0,1 cup,0
3,1004,"CHEESE,BLUE",42.41,353,21.40,28.74,2.34,0.0,0.50,528,...,2.4,18.669,7.778,0.800,75,28.35,1 oz,17.0,1 cubic inch,0
4,1005,"CHEESE,BRICK",41.11,371,23.24,29.68,2.79,0.0,0.51,674,...,2.5,18.764,8.598,0.784,94,132.00,"1 cup, diced",113.0,"1 cup, shredded",0


In [7]:
X_text = df["Shrt_Desc"].values
X_text

array(['BUTTER,WITH SALT', 'BUTTER,WHIPPED,W/ SALT',
       'BUTTER OIL,ANHYDROUS', ..., 'SYRUP,CANE', 'SNAIL,RAW',
       'TURTLE,GREEN,RAW'], dtype=object)

In [8]:
cv = make_pipeline(
    CountVectorizer(
            ngram_range=(3, 7),
            analyzer="char"
        ),
    Normalizer()
)

In [9]:
cv.fit(X_text)

Pipeline(memory=None,
         steps=[('countvectorizer',
                 CountVectorizer(analyzer='char', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(3, 7), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)),
                ('normalizer', Normalizer(copy=True, norm='l2'))],
         verbose=False)

In [10]:
X = cv.transform(X_text)

In [11]:
X_term = cv.transform(["choclte chip sookies"])
# text_matrix = X_norm.todense()

In [12]:
simularities = cosine_similarity(X_term, X)

In [13]:
simularities[0]

array([0.      , 0.016265, 0.      , ..., 0.      , 0.      , 0.      ])

In [14]:
k = 10
result = np.sort(np.argpartition(simularities[0], len(simularities[0]) - k)[-k:])
#np.sort(np.argpartition(arr, len(arr) - k)[-k:])

In [15]:
#idxmax = np.argmax(simularities[0])

In [17]:
df.loc[result]["Shrt_Desc"]
# df.iloc?[0]["S?hrt_Desc"]

5675                            COOKIES,CHOC CHIP,DRY MIX
5937    ARCHWAY Home Style Cookies, Chocolate Chip Ice...
7835                        FAMOUS AMOS,CHOC CHIP COOKIES
7836                  FAMOUS AMOS,CHOC CHIP PECAN COOKIES
7863          KEEBLER,CHIPS DELUXE,MINI CHOC CHIP COOKIES
7864       KEEBLER,CHIPS DELUXE,OATMEAL CHOC CHIP COOKIES
7867       KEEBLER,CHIPS DELUXE,RAINBOW CHOC CHIP COOKIES
7998                           MOTHER'S,CHOC CHIP COOKIES
8079                  MURRAY,SUGAR FREE,CHOC CHIP COOKIES
8130            COOKIES,CHOC CHIP SNDWCH,W/ CREME FILLING
Name: Shrt_Desc, dtype: object

In [18]:
# Display info of the first item matching the search string
# def search(term):
#     X_term = cv.transform([term])
#     simularities = cosine_similarity(X_term, X)
#     idxmax = np.argmax(simularities[0])
#     return df.loc[idxmax]
# if __name__ == "__main__":
#     term = "doritos"
#     print(search(term))

In [19]:
# Recommend 5 items similar to text in search string
def advanced_search(term):
    X_term = cv.transform([term])
    simularities = cosine_similarity(X_term, X)
    k = 10
    result = np.sort(np.argpartition(simularities[0], len(simularities[0]) - k)[-k:])

    #idxmax = np.arange(start=simularities[0], stop = simularities[5] , step=1)
    return df.loc[result]
if __name__ == "__main__":
    term = "egg"
    term1 = "choclate cooies mnt"
    print(advanced_search(term)["Shrt_Desc"])

55                                                 EGGNOG
121                                         EGG,WHL,DRIED
125                                        EGG,YOLK,DRIED
151                                       EGG,WHITE,DRIED
2961                                         EGGPLANT,RAW
5564                                           BAGELS,EGG
5583                                            BREAD,EGG
5831                                     ROLLS,DINNER,EGG
6043    KELLOGG'S,EGGO,BISCUIT SCRAMBLERS,BACON,EGG & CHS
6044          KELLOGG'S,EGGO,BISCUIT SCRAMBLERS,EGG & CHS
Name: Shrt_Desc, dtype: object


In [20]:
#X_term = cv.transform(["butter"])

In [21]:
#simularities = cosine_similarity(X_term, X)

In [22]:
#idxmax = np.argmax(simularities[0])

In [19]:
#df.loc[idxmax]